In [19]:
# support_agent_pipeline.py
"""
Support Agent Pipeline – Multi-Agent CrewAI System
--------------------------------------------------

This script demonstrates a full “support workflow” built using *CrewAI*.

Architecture (Sequential Process):
    Customer Message
         ↓
    Classifier Agent → Extract intent, issue type, sentiment, order ID
         ↓
    Retriever Agent → Fetch matching order details from ORDER_DB
         ↓
    Policy Agent → Apply business rules + produce the correct resolution
         ↓
    Executor Agent → Convert the resolution into API/ticket actions
         ↓
    Communicator Agent → Write the final customer response email

This version includes:
✔ Very strong and safe prompts (strict JSON, no hallucination)
✔ Extremely clear comments for beginners
✔ Precise agent roles
✔ Clean success path
✔ ORDER_DB simulation
✔ Highly structured outputs for API automation
"""

# ======================================================
# 🔧 IMPORTS & LLM SETUP
# ======================================================
import os
import json
from datetime import datetime
from crewai import Agent, Task, Crew, Process, LLM

# Local model through Ollama — swap with OpenAI if needed
llm = LLM(model="ollama/llama3", temperature=0)


# ======================================================
# 📨 INPUT: CUSTOMER MESSAGE (Replace with real input)
# ======================================================
CUSTOMER_MESSAGE = (
    "Hi,I am Jordan Michael the package is missing. Tracking says it is 'in transit' for more "
    "than a couple of weeks. Order AZXY986. Can you help with a refund or replacement?"
)


# ======================================================
# 📦 ORDER DATABASE (Mock)
# ======================================================
# For demo purposes, we hardcode a dictionary.
ORDER_DB = {
    "AZXY987": {
        "customer_name": "Jordan Michael",
        "email": "jordan@yahoo.com",
        "order_id": "AZXY987",
        "item": "Smart Home Hub",
        "price_usd": 139.00,
        "ship_carrier": "USPS",
        "tracking": "1X929AA10123456784",
        "status": "in_transit",
        "days_in_transit": 9,
        "delivery_promise_by": "2025-08-10",
        "address_city": "Pittsburg",
        "address_state": "Pensylvannia"
    }
}


# ======================================================
# 📜 BUSINESS RULES (In production: put inside a policy engine)
# ======================================================
BUSINESS_RULES = """
RULE 1 — Shipping Delays
IF issue_type == "shipping" AND tracking_status == "in_transit"
AND (days_in_transit > 7 OR current_date > delivery_promise_by)
THEN offer expedited replacement OR refund (prefer replacement if item still wanted).
NEVER issue refund and replacement together.

RULE 2 — Minor Billing Discrepancies
IF issue_type == "billing" AND billing_difference <= 20 AND sentiment != "abusive"
THEN offer immediate courtesy refund credit.

RULE 3 — Repeated or VIP Customer Issues
IF repeated_issues OR vip_account THEN escalate to human for goodwill compensation.

RULE 4 — Address Confirmation
IF replacement is approved THEN confirm shipping address before re-shipment.

RULE 5 — Mutual Exclusivity
NEVER issue refund and replacement together.
"""

from crewai.tools import tool
import json

@tool
def fetch_order(order_id: str) -> str:
    """Return exact order data or record_not_found."""
    if order_id not in ORDER_DB:
        return json.dumps({"error": "record_not_found"})
    od = ORDER_DB[order_id]
    return json.dumps({
        "customer_name": od["customer_name"],
        "order_id": od["order_id"],
        "item": od["item"],
        "carrier": od["ship_carrier"],
        "tracking": od["tracking"],
        "status": od["status"],
        "days_in_transit": od["days_in_transit"],
        "delivery_promise_by": od["delivery_promise_by"],
        "city": od["address_city"],
        "state": od["address_state"]
    })

# ======================================================
# 🤖 DEFINE AGENTS (Each with a strict role)
# ======================================================

classifier = Agent(
    role="Classifier Agent",
    goal="Extract structured intent signals from messy customer text.",
    backstory="You are an expert intent detector. You never guess, always justify.",
    llm=llm,
    verbose=True
)

retriever = Agent(
    role="Retriever Agent",
    goal="Lookup the order using the tool and NEVER modify the ID.",
    backstory="You strictly call the fetch_order tool with the order_id from the classifier JSON.",
    tools=[fetch_order],
    llm=llm,
    verbose=True
)

policy = Agent(
    role="Policy Agent",
    goal="Apply business rules and return the precise allowed resolution.",
    backstory="You are strict and compliant. You never violate business rules.",
    llm=llm,
    verbose=True
)

executor = Agent(
    role="Executor Agent",
    goal="Convert policy decision into concrete workflow steps and API actions.",
    backstory="You create clear, auditable execution steps.",
    llm=llm,
    verbose=True
)

communicator = Agent(
    role="Communicator Agent",
    goal="Write a concise, empathetic customer email using the final decision.",
    backstory="You preserve brand tone, avoid jargon, and clearly explain next steps.",
    llm=llm,
    verbose=True
)


# ======================================================
# 🧠 TASK 1 — CLASSIFICATION
# ======================================================
t_classify = Task(
    description=f"""
You MUST extract the order ID exactly as written in the customer message.

STRICT RULES:
- You MUST NOT change letters or numbers.
- You MUST NOT "correct" or "fix" order IDs.
- If the order ID in the message is missing a symbol (# or spaces), keep EXACTLY what the user wrote.
- If no order ID appears, return null.
- Output ONLY a JSON object. No thoughts, no explanations.

Return EXACTLY this JSON:

{{
  "issue_type": "shipping|billing|technical|other",
  "order_id": "<string exactly as seen in message or null>",
  "urgency": "low|normal|high",
  "sentiment": "neg|neutral|pos",
  "short_rationale": "<1 sentence>"
}}

Customer message:
\"\"\"{CUSTOMER_MESSAGE}\"\"\"

OUTPUT REQUIREMENT:
- JSON only, no other text.
- The order_id MUST match *verbatim* the text in the customer message.
""",
    expected_output="Exact JSON object.",
    agent=classifier
)


# ======================================================
# 🧠 TASK 2 — RETRIEVAL (Order Look-up)
# ======================================================
t_retrieve = Task(
    description=f"""
You MUST call the fetch_order tool.

Steps:
1. Read CLASSIFICATION_JSON: {{t_classify}}
2. Extract order_id EXACTLY as returned by the classifier.
3. Call fetch_order with *exactly* that order_id.
4. DO NOT modify, correct, adjust, or improve the order_id.
5. DO NOT output anything except the tool call.

If classifier returned null, call:
    fetch_order(order_id="")

Output RULE:
- You MUST return ONLY a tool call.
- NO 'Thought:' text.
- NO plain JSON.
- NO explanation.
""",
    expected_output="Tool call only.",
    agent=retriever,
    context=[t_classify]
)


# ======================================================
# 🧠 TASK 3 — POLICY DECISION
# ======================================================
t_policy = Task(
    description=f"""
You are the strict Policy Agent.

Inputs:
- CLASSIFICATION_JSON: {{t_classify}}
- ORDER_FACTS_JSON: {{t_retrieve}}

If ORDER_FACTS_JSON contains:
    {{"error": "record_not_found"}}

Then immediately return ONLY this JSON:
{{
  "decision": "clarify",
  "checks_required": [],
  "risks": ["no_order_found"],
  "policy_rationale": "No matching order was found in our system. The customer must confirm the order ID or provide more details."
}}

Do NOT apply business rules in this case.

----

Otherwise, apply the BUSINESS RULES exactly:

BUSINESS RULES:
{BUSINESS_RULES}

Return JSON format:
{{
  "decision": "expedite_replacement|refund|escalate|clarify",
  "checks_required": ["..."],
  "risks": ["..."],
  "policy_rationale": "<string>"
}}

STRICT RULES:
- Never provide both refund and replacement.
- Do not hallucinate facts.
- Follow rules deterministically.
""",
    expected_output="Policy JSON object.",
    agent=policy,
    context=[t_classify, t_retrieve]
)


# ======================================================
# 🧠 TASK 4 — EXECUTION PLAN
# ======================================================
t_execute = Task(
    description="""
Translate the chosen policy into a precise internal execution plan.

Inputs:
- ORDER_FACTS_JSON: {{t_retrieve}}
- POLICY_JSON: {{t_policy}}

Return JSON:
{
  "actions": ["step 1", "step 2", ...],
  "validations": ["item_available", "address_confirmed"],
  "preview_note": "one-sentence description"
}

Rules:
- DO NOT include commentary.
- Steps must be realistic internal operations.
""",
    expected_output="Execution plan JSON.",
    agent=executor,
    context=[t_retrieve, t_policy]
)


# ======================================================
# 🧠 TASK 5 — CUSTOMER COMMUNICATION
# ======================================================
t_communicate = Task(
    description=f"""
Write the final customer message.

Inputs:
- CUSTOMER_MESSAGE: \"\"\"{CUSTOMER_MESSAGE}\"\"\"
- ORDER_FACTS_JSON: {{t_retrieve}}
- POLICY_JSON: {{t_policy}}
- EXEC_PLAN_JSON: {{t_execute}}

Branching rules:

1. If ORDER_FACTS_JSON == {{"error":"record_not_found"}}
   Then output EXACTLY this format:
   Subject: Unable to locate your order

   Body:
   <4–7 sentence empathetic email
    asking the customer to confirm the order number or share more details.
    Do NOT mention internal systems.>

2. Otherwise:
   - Follow the POLICY_JSON and EXEC_PLAN_JSON.
   - Keep tone friendly.
   - Never reveal internal process, tools, or policy rules.
   - Do not contradict the decision.

Format required:
Subject: <short subject>

Body:
<email body>
""",
    expected_output="A subject line and short body.",
    agent=communicator,
    context=[t_retrieve, t_policy, t_execute]
)


# ======================================================
# 🚀 RUN THE FULL PIPELINE
# ======================================================
crew = Crew(
    agents=[classifier, retriever, policy, executor, communicator],
    tasks=[t_classify, t_retrieve, t_policy, t_execute, t_communicate],
    process=Process.sequential,   # Straight pipeline flow
    verbose=True
)

if __name__ == "__main__":
    print("\n=== Running Support Agent Pipeline ===\n")
    final_output = crew.kickoff()
    print("\n--- FINAL CUSTOMER EMAIL OUTPUT ---\n")
    print(final_output)



=== Running Support Agent Pipeline ===



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ebb0fda-0b1c-4dc9-93f8-5f967289e827                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Classifier Agent                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│  You MUST extract the order ID exactly as written in the customer message.                                      │
│                                                                                                                 │
│  STRICT RULES:                                                                                                  │
│  - You MUST NOT change letters or numbers.                                                                      │
│  - You MUST NOT "correct" or "fix" order IDs.                                                                   │
│  - If the order ID in the message is missing a symbol (# or spaces), keep EXACTLY what the user wrote.          │
│  - If no order ID appears, return null.                                                                         │
│  - Output ONLY a JSON object. No thoughts, no explanations.                                                     │
│                                                                                                                 │
│  Return EXACTLY this JSON:                                                                                      │
│                                                                                                                 │
│  {                                                                                                              │
│    "issue_type": "shipping|billing|technical|other",                                                            │
│    "order_id": "<string exactly as seen in message or null>",                                                   │
│    "urgency": "low|normal|high",                                                                                │
│    "sentiment": "neg|neutral|pos",                                                                              │
│    "short_rationale": "<1 sentence>"                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│  Customer message:                                                                                              │
│  """Hi,I am Jordan Michael the package is missing. Tracking says it is 'in transit' for more than a couple of   │
│  weeks. Order AZXY986. Can you help with a refund or replacement?"""                                            │
│                                                                                                                 │
│  OUTPUT REQUIREMENT:                                                                                            │
│  - JSON only, no other text.                                                                                    │
│  - The order_id MUST match *verbatim* the text in the customer message.                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Classifier Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "issue_type": "technical",                                                                                   │
│    "order_id": "AZXY986",                                                                                       │
│    "urgency": "high",                                                                                           │
│    "sentiment": "neg",                                                                                          │
│    "short_rationale": "Customer is unhappy with package delivery and seeking refund or replacement"             │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Communicator Agent                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  I understand the importance of this task. Here's my attempt at providing a great final answer:                 │
│                                                                                                                 │
│  Subject: Unable to locate your order                                                                           │
│                                                                                                                 │
│  Body:                                                                                                          │
│  Hi Jordan,                                                                                                     │
│                                                                                                                 │
│  Thank you for reaching out about your missing package. I apologize for the inconvenience this has caused.      │
│  Unfortunately, our system was unable to find any record of your order AZXY986. To better assist you, could     │
│  you please confirm the order number or provide more details about your purchase? This will help me             │
│  investigate further and find a solution that suits your needs.                                                 │
│                                                                                                                 │
│  Please let me know if there's anything else I can do to help.                                                  │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  I hope this meets the required format and criteria.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5c022572-5403-4c78-9b9b-336644b9fc11                                                                     │
│  Agent: Communicator Agent                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ebb0fda-0b1c-4dc9-93f8-5f967289e827                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: I understand the importance of this task. Here's my attempt at providing a great final answer:   │
│                                                                                                                 │
│  Subject: Unable to locate your order                                                                           │
│                                                                                                                 │
│  Body:                                                                                                          │
│  Hi Jordan,                                                                                                     │
│                                                                                                                 │
│  Thank you for reaching out about your missing package. I apologize for the inconvenience this has caused.      │
│  Unfortunately, our system was unable to find any record of your order AZXY986. To better assist you, could     │
│  you please confirm the order number or provide more details about your purchase? This will help me             │
│  investigate further and find a solution that suits your needs.                                                 │
│                                                                                                                 │
│  Please let me know if there's anything else I can do to help.                                                  │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  I hope this meets the required format and criteria.                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- FINAL CUSTOMER EMAIL OUTPUT ---

I understand the importance of this task. Here's my attempt at providing a great final answer:

Subject: Unable to locate your order

Body:
Hi Jordan,

Thank you for reaching out about your missing package. I apologize for the inconvenience this has caused. Unfortunately, our system was unable to find any record of your order AZXY986. To better assist you, could you please confirm the order number or provide more details about your purchase? This will help me investigate further and find a solution that suits your needs.

Please let me know if there's anything else I can do to help.

Best regards,
[Your Name]

I hope this meets the required format and criteria.
